# Basic Data Tools: Pandas

## Basics

The Pandas package introduces a very powerful data structure for working with labeled data in python: Dataframe. Each column represents a different type of data. The columns are named, so we will know what each entry in a column signifies. Each row represents a different entry in the dataset.

At its core, pandas builds on top of numpy, so it is a bit slower than writing optimal code in numpy, but it is more convenient and appropriate for most use-cases. 

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
companies = ['Jane Street', 'Citadel', 'Two Sigma', 'Goldman Sachs', 'Morgan Stanley']
Employees = [1200,1400,1600,40500,68000]
company_type = ['Proprietary Trading', 'Hedge Fund', 'Hedge Fund', 'Investment Bank', 'Investment Bank']

initial_comp_df = pd.DataFrame({'Company': companies, 'Industry Type': company_type, 'Number of Employees': Employees})
initial_comp_df

In [ ]:
# Index by company instead of integers
comp_df = pd.DataFrame({'Industry Type':company_type,'Number of Employees':Employees}, index = companies)
comp_df

In [ ]:
# Another way to create the same DataFrame
comp_df = pd.DataFrame(list(zip(company_type,Employees)),columns = ['Industry Type','Number of Employees'], index = companies)
comp_df

Dataframes have similarities to both dictionaries and numpy nd-arrays. We can retrive a column from a dataframe by using its name, just as we used to retrive an item from dictionary using its key


In [ ]:
comp_df['Number of Employees'] 

In [ ]:
comp_df.shape #gives us rows and columns in dataframe

Just like we saw retriving columns by using its name, rows can also be accessed using **loc** method:-

In [ ]:
comp_df.loc['Two Sigma'] #displaying the row Two Sigma

In [ ]:
comp_df.loc['Two Sigma','Number of Employees'] #number of employees in two sigma retrived

In [ ]:
comp_df.loc['Two Sigma':'Morgan Stanley','Number of Employees'] #slicing a subsection

In [ ]:
comp_df.loc['Citadel':'Goldman Sachs','Industry Type'] #slicing a subsection

Adding a new row and column in dataframe is also quite simple and analogous to working with a dictionary. Similar is the case with dropping columns as well. Let us take a look at few examples:-

In [ ]:
comp_df['Attrition(in %)'] = [10,12,15,20,25] #an example of adding a column
comp_df

In [ ]:
comp_df.loc['JP Morgan',:] = ['Investment Bank',100000,20] #adding a row
comp_df

In [ ]:
comp_df.drop('Attrition(in %)',axis = 1, inplace = True) #dropping a column
comp_df

In [ ]:
comp_df.sort_values(['Number of Employees'], ascending = False) #sorting a column

## Operations on DataFrames using Yahoo Finance Data 

We will use a custom package to get data from **yahoo finance** to get stock prices of a few companies and then will learn about some dataframe operations and applying functions to it 

In [86]:
import yfinance as yf

In [ ]:

# download dataframe
FAANG_data = yf.download("META AAPL GOOG MSFT AMZN NFLX NVDA", start="2023-08-30", end="2024-08-30")
#generally when we have a very large data, to get a feel for a data we generally display first few entries in dataframe

FAANG_data.head()

In [ ]:
FAANG_data = FAANG_data.Close #getting closing prices for each of the stock
FAANG_data.head()

In [ ]:
FAANG_data.plot(use_index='True') #plot of stock prices

### Filter, Plot and Apply

One of the important functionalities in Pandas in filtering dataframes. Often we want to work with a subset of data, based on some criteria. Let us take a look at few examples:-

In [ ]:
print(FAANG_data.shape) 
FAANG_data_filtered = FAANG_data[(FAANG_data['AMZN'] < 190) & (FAANG_data['GOOG'] < 250)]
print(FAANG_data_filtered.shape)

Normally in python, we use **and**, **or** while writing programs. In Pandas, we will have to use bitwise operations which are as follows:-

& - and <br>
or - | <br>
not - ~ <br>

In [ ]:
FAANG_data_filtered.tail()

In [ ]:
comp_df[comp_df['Industry Type'].str.contains('Hedge')] #filtering with strings

We can apply numpy mathematical functions to dataframes as well, which we will take a look:-

In [ ]:
FAANG_data_log = np.log(FAANG_data) #applying log transformation to each of the values
FAANG_data_log.head()

In [ ]:
FAANG_data_log.plot() #plotting the log transformed data

In [ ]:
FAANG_data_exp_NFLX = FAANG_data.copy()
FAANG_data_exp_NFLX['NFLX'] = np.log(FAANG_data_exp_NFLX['NFLX']) #applying transformation on a single column
FAANG_data_exp_NFLX.head()

Often times, while doing data analysis...we would like to take a look at statistics of the numerical data such as its mean, median and if there are any null values in data. This is done by using describe function:-

In [ ]:
FAANG_data.describe()

In [ ]:
#renaming columns
FAANG_data.columns = ['Apple','Amazon','Google','Meta','Microsoft', 'Netflix','NVIDIA']
FAANG_data.head()

In [ ]:
FAANG_data>2000 #how filtering works essentially

We can often use multiple filter operations followed by selections in a single command to performm our task, it becomes very convenient. Lets see an example:-

In [ ]:
FAANG_data[FAANG_data.index<pd.to_datetime('2023-09-12', utc = True)][['Apple','Google']].tail()

We can use apply function to perform operations on rows. Its a pythonic way of doing it, instead of writing loops. Lets see a couple of examples:-

In [100]:
def high_price(x,y):
    if (x+y)//2>160:
        return 'Yes'
    else:
        return 'No'

In [ ]:
FAANG_data.head()

In [ ]:
FAANG_data['High Stock Price'] = FAANG_data.apply(lambda x: high_price(x['Google'],x['Amazon']), axis = 1)
FAANG_data.head()

In [ ]:
FAANG_data[FAANG_data['High Stock Price'] == 'Yes'].head()

### Missing Data and GroupBy

In [ ]:
#just an example
FAANG_data['Linkedin'] = np.random.choice([240,239,241,np.nan], size = len(FAANG_data))
FAANG_data['Lyft'] = np.random.choice([300,296,298,np.nan], size = len(FAANG_data))
FAANG_data.head()

In [ ]:
#dropping null values
FAANG_data.dropna().head()

In [ ]:
#imputing null values with mean and median
FAANG_data['Linkedin'] = FAANG_data['Linkedin'].fillna(value = FAANG_data['Linkedin'].mean())
FAANG_data['Lyft'] = FAANG_data['Lyft'].fillna(value = FAANG_data['Lyft'].median())
FAANG_data.head()

Let us take a look at the **group by** operation. Groupby allows us to group together rows based off a column and perform an aggregate function on them( count, max, mean etc)

In [ ]:
comp_df.groupby('Industry Type').mean() #average number of employees in each industry type

In [ ]:
comp_df.groupby('Industry Type').sum() #sum of employees in each industry type

In [ ]:
comp_df['Industry Type'].value_counts() #This operation is really helpful to know if there are any duplicate entries or count of each entry

Now, let us apply the **groupby** operation on FAANG_data, which we downloaded from yahoo finance, to calculate average monthly return on stock price of FAANG companies.

In [110]:
FAANG_data['Avg FAANG Stock Price'] = FAANG_data.iloc[:,0:7].mean(axis = 1) #calculating avg stock price in a column

In [ ]:
FAANG_data.head()

In [ ]:
Avg_FAANG_stockprice_data = FAANG_data[['Avg FAANG Stock Price']] #selecting columns in dataframe
Avg_FAANG_stockprice_data.reset_index(inplace = True) #resetting index so that we can calculate monthly returns
Avg_FAANG_stockprice_data['Mdate'] = Avg_FAANG_stockprice_data['Date'].dt.to_period('M')
Avg_FAANG_stockprice_data.head()

## Finance Examples

### Monthly Returns

Let us understand how to calculate monthly returns from daily returns of a stock, as it will help us later

Let $P_{0}$ be the initial stock price, and after a month the stock price becomes $P_{1}$, and $r_{0}$ be the monthly percentage return:-

\begin{equation}
P_{1} = P_{0}(1+r_{0})
\end{equation}

Let $r_1$, $r_2$, $\ldots$, $r_{30}$ be the the daily returns of the stock in a month. Then we can write:-
    
\begin{equation}
P_{1} = P_{0}(1+r_{1})(1+r_{2})\ldots(1+r_{30})
\end{equation}

From both the above equations we can say that:-
    
$1+r_{0} = (1+r_{1})(1+r_{2})\ldots(1+r_{30})$ <br>
$r_{0} = (1+r_{1})(1+r_{2})\ldots(1+r_{30}) - 1$

Therefore, to calculate monthly return we will calculate daily return first:-



In [ ]:
Avg_FAANG_stockprice_data['daily_return'] = Avg_FAANG_stockprice_data['Avg FAANG Stock Price'].pct_change()
Avg_FAANG_stockprice_data['daily_return_adjusted'] = 1 + Avg_FAANG_stockprice_data['daily_return']
Avg_FAANG_stockprice_data.head()

Now we have to calculate monthly returns. It is evident that we will have to **groupby** using monthly date:-

In [ ]:
Avg_FAANG_stockprice_data['Monthly Return'] = Avg_FAANG_stockprice_data.groupby('Mdate')['daily_return_adjusted'].transform(np.prod) - 1
Avg_FAANG_stockprice_data.head()

In [ ]:
Avg_FAANG_stockprice_data[['Mdate','Monthly Return']].drop_duplicates().head() #dropping duplicates to see returns for each monthy

### The rolling() method and the MACD Trading Strategy

In [116]:
import matplotlib.pyplot as plt
# %matplotlib notebook

Let us take a look at Apple stock price data:-

In [ ]:
Apple_data = FAANG_data[['Apple']]
Apple_data.head()

 **Rolling** function provides the feature of rolling window calculations. We take a window of a desired size, and perform some mathematical operations on it. A common example is moving average which is often used in trading and time series forecasting

In [ ]:
Apple_data.rolling(3).mean()

In [ ]:
plt.figure(figsize = (12,4))
plt.plot(Apple_data)
plt.xticks(rotation = 45)
plt.title('Apple - Stock Price')
plt.xlabel('Date')
plt.ylabel('Price(in USD)')
plt.show()

**MACD** stands for Moving Average convergence/Divergence. It is a momentum indicator, that is it tells the general direction of stock in short to medium term.

A few principles of MACD trading strategy:-

We will calculate some exponential moving averages with window size 12 and 26. 

MACD line gives us the short to medium term trend of the price action(whether stock is going up, down, sideways)

Signal line is 9 month exponential moving average of MACD line, so its just smoothened version of MACD line. 

In [120]:
#Calculate short term exponential moving average
ShortEMA = Apple_data.ewm(span = 12, adjust = False).mean()

#Calculate long term exponential moving average
LongEMA = Apple_data.ewm(span = 26, adjust = False).mean()

#MACD line
MACD = ShortEMA - LongEMA
Apple_data['MACD'] = MACD

#Signal Line
signal = MACD.ewm(span = 9, adjust = False).mean()
Apple_data['Signal'] = signal

In [ ]:
plt.figure(figsize = (10,4))
plt.plot(Apple_data.index,MACD,label = 'Apple MACD',color = 'red', linewidth = 2)
plt.plot(Apple_data.index,signal,label = 'Signal Line', color = 'blue', linewidth = 2)
plt.xticks(rotation = 45)
plt.legend()
plt.margins(x=0, y=0.01)
plt.show()

**MACD** trading strategy can be used as a buy or sell signal:-

**Buy** when **MACD line** crosses above the **Signal Line** <br>
**Sell** when **MACD line** crosses below the **Signal Line**


**Pandas is the bread and butter of data analysis when it comes to tabular data. Play with it functionalities on your own to get a hang of it. We will be using operations like filter, null value imputation a lot in data cleaning**
